##  District Summary

###  analysis of the several factors on the passing grade

1. school type: Charter schools showed the much better overall passing % (95% vs. 73%)
especially, there was a big difference in math area (94% vs.66%)
2. school size: Definitely, smaller size of school outperformed the larger size of school 
such as 94% vs 76%. Overall, there was no difference in the performance between small and the meium size of school, showing that large size above 2.000 students mad a big difference in their performance.
3. The charter school all seem to have a smaller size under 2000, while the district schools all show big size of the students as 2000-5000. Similarly, all the 5 top school were charter schools meanwhile all the bottom 5 schools were district schools.
4. budget per student: Surprisingly, it doesn't seem to affect the students' performance, at all.
5, Grade (9-12th): The grade doesn't sesem to ahve any effect on the performance. Rather, thier average score was swayed by individual school's atmosphere. Especially, math score was affected greatly than reading. 


In [1]:
# dependencies
import pandas as pd
import numpy  as np


In [2]:
# read the csv files into df1 and df2
school_data  = "schools_complete.csv" 
student_data = "students_complete.csv"

df1 = pd.read_csv(school_data) 
df2 = pd.read_csv(student_data)


In [3]:
# count the number of schools in the district
t_schools =df1['School ID'].count()

# total number of students and budget of the district
t_students=df1['size'].sum()
t_budget  =format(df1['budget'].sum(),'.2f')

# average math and reading score in the district
avg_math  =format(df2['math_score'].sum()/t_students,'.6f')
avg_read  =format(df2['reading_score'].sum()/t_students,'.5f')


In [4]:
# calculate the passing % of math, reading and combined scores of the district
p_math =len(df2[df2['math_score']>=70])/t_students
passRate_math =format(p_math*100,'.6f')

p_read =len(df2[df2['reading_score']>=70])/t_students
passRate_read =format(p_read*100,'.6f')

Overall_passRate=format(((p_math + p_read)/2)*100,'.6f')


In [5]:
# make a summary table (formatting,deleting and renaming the variables)
district_summary= pd.DataFrame({"Total Schools":[t_schools] ,
                           "Total Students":[t_students],
                           "Total Budget": str(t_budget),
                           "Average Math Score": [avg_math],
                           "Average Reading Score": [avg_read],                          
                           "% Passing Math": [passRate_math], 
                           "% Passing Reading": [passRate_read] ,
                           "% Overall Passing Rate": [Overall_passRate]})
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


##  School Summary

In [6]:
# calculate the budget per student
df1['budget per student']=df1['budget']/df1['size']


In [7]:
# calculate the average(mean) of math_score & reading_score per each school
g_df2 = df2.groupby("school_name")
mean_df  = g_df2['math_score','reading_score'].mean()
mean_df  = mean_df.rename(columns={'reading_score':'average reading score','math_score':'average math score'})


In [8]:
# choose the students with math_score>=70
m_70 = df2[df2['math_score']>=70][['school_name', 'math_score']]
m_70 = m_70.rename(columns={'math_score':'math_ge 70'})

# count the number of the students with math_score >=70 per each school
g_m70= m_70.groupby(["school_name"])
count_m70 = g_m70.count()
count_m70 = count_m70.rename(columns={'math_ge 70':'math_70 count'})


In [9]:
# choose the students with reading_score>=70
r_70 =df2[df2['reading_score']>=70][['school_name', 'reading_score']]
r_70 = r_70.rename(columns={'reading_score':'reading_ge 70'})

# count the number of the students with reading_score >=70 per each school
g_r70=r_70.groupby(["school_name"])
count_r70 = g_r70.count()
count_r70 = count_r70.rename(columns={'reading_ge 70':'reading_70 count'})


In [10]:
# merge mean data, count data with the school data
df_a = pd.merge(df1, mean_df, how="outer",on=["school_name"])
df_b = pd.merge(df_a, count_r70, how="outer",on=["school_name"])
df_d = pd.merge(df_b, count_m70, how="outer",on=["school_name"])


In [11]:
# calculate the passing % 
df_d['% passing math'] = (df_d['math_70 count']/df_d['size'])*100
df_d['% passing reading'] = (df_d['reading_70 count']/df_d['size'])*100
df_d['% Overall passing Rate'] =(df_d['% passing math']+df_d['% passing reading'])/2

# delete some unneccesary variables and rename some variables
df_d.drop(['School ID','math_70 count', 'reading_70 count'], axis=1, inplace=True)
df_d.rename(columns={'type':'school type','size':'total students','budget':'total budget'}, inplace=True)

# delete the index and use school_name as a new index
df3 = df_d.set_index("school_name")

##  Top Performing Schools (By Passying Rate)

In [12]:
df_top=df3.sort_values('% Overall passing Rate', ascending=False)
df_top["total budget"] = df_top["total budget"].astype(float).map("${:,.2f}".format)
df_top["budget per student"] = df_top["budget per student"].astype(float).map("${:,.2f}".format)
df_top.head()

,school type,total students,total budget,budget per student,average math score,average reading score,% passing math,% passing reading,% Overall passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


##  Bottom Performing Schools (By Passying Rate)

In [13]:
df_bottom=df3.sort_values('% Overall passing Rate', ascending=True)
df_bottom["total budget"] = df_bottom["total budget"].astype(float).map("${:,.2f}".format)
df_bottom["budget per student"] = df_bottom["budget per student"].astype(float).map("${:,.2f}".format)
df_bottom.head()

,school type,total students,total budget,budget per student,average math score,average reading score,% passing math,% passing reading,% Overall passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


##  Math Scores by Grade

In [14]:
# Create the math scores for each grade and school
col_9 =df2[df2['grade']=='9th'][['school_name', 'math_score']]
col_9.rename(columns={'math_score':'9th'},inplace=True)

col_10=df2[df2['grade']=='10th'][[ 'school_name','math_score']]
col_10.rename(columns={'math_score':'10th'},inplace=True)

col_11=df2[df2['grade']=='11th'][[ 'school_name', 'math_score']]
col_11.rename(columns={'math_score':'11th'},inplace=True)

col_12=df2[df2['grade']=='12th'][[ 'school_name', 'math_score']]
col_12.rename(columns={'math_score':'12th'},inplace=True)


In [15]:
# calculate average math for each grade per school
g_9=col_9.groupby("school_name")
gm_9= g_9['school_name','9th'].mean()

g_10=col_10.groupby("school_name")
gm_10= g_10['school_name','10th'].mean() 

g_11=col_11.groupby("school_name")
gm_11= g_11['school_name','11th'].mean()

g_12=col_12.groupby("school_name")
gm_12= g_12['school_name','12th'].mean()


In [16]:
# merge the average math for each school by the grades (9th-12th)
df_m1 = pd.merge(gm_9, gm_10, how="outer", on=["school_name"])
df_m2 = pd.merge(gm_11,gm_12, how="outer", on=["school_name"])
df_m  = pd.merge(df_m1,df_m2, how="outer",  on=["school_name"])
df_m

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


##  Reading Score by Grade

In [17]:
# Create the reading scores for each grade and school
col_9 =df2[df2['grade']=='9th'][['school_name', 'reading_score']]
col_9.rename(columns={'reading_score':'9th'},inplace=True)

col_10=df2[df2['grade']=='10th'][[ 'school_name','reading_score']]
col_10.rename(columns={'reading_score':'10th'},inplace=True)

col_11=df2[df2['grade']=='11th'][[ 'school_name', 'reading_score']]
col_11.rename(columns={'reading_score':'11th'},inplace=True)

col_12=df2[df2['grade']=='12th'][[ 'school_name', 'reading_score']]
col_12.rename(columns={'reading_score':'12th'},inplace=True)


In [18]:
# calculate average reading score for each grade per school
g_9=col_9.groupby("school_name")
gm_9= g_9['school_name','9th'].mean()
 
g_10=col_10.groupby("school_name")
gm_10= g_10['school_name','10th'].mean()     

g_11=col_11.groupby("school_name")
gm_11= g_11['school_name','11th'].mean()

g_12=col_12.groupby("school_name")
gm_12= g_12['school_name','12th'].mean()


In [19]:
# merge the average reading score for each school by the grades (9th-12th)
df_r1 = pd.merge(gm_9, gm_10, how="outer", on=["school_name"])
df_r2 = pd.merge(gm_11,gm_12, how="outer", on=["school_name"])
df_r  = pd.merge(df_r1, df_r2, how="outer",  on=["school_name"])
df_r

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [20]:
# define the bins and the name of the each bin group according to the school spending
sp_bins = [0, 585, 615, 645, 675]
sp_gp = ["<$585", "$585-615", "$615-645", "$645-675"]   


In [21]:
# allocate the schools into the bin groups by the 'budget per student' (school spending)
df3["spending_range"] = pd.cut(df3["budget per student"], sp_bins, labels=sp_gp)


In [22]:
# calculate the average scores and the passing scores by the school spending
gdf_sp=df3.groupby("spending_range")
m_df_sp= gdf_sp.mean()
m_df_sp= m_df_sp.rename(columns={'avg_read':'Average Reading Score','avg_math':'Average Math Score'})
m_df_sp.drop(['total students', 'total budget','budget per student'],axis=1,inplace=True)
m_df_sp

,average math score,average reading score,% passing math,% passing reading,% Overall passing Rate
spending_range,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


##  Scores by School Size

In [23]:
# define the bins and the name of the each bin group according to the school size
s_bins = [0, 1000, 2000, 5000]
s_gp = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [24]:
# allocate the schools into the bin groups by the school size
df3["school size"] = pd.cut(df3["total students"], s_bins, labels=s_gp)


In [25]:
# calculate the average scores and the passing scores by the school size
gdf_s=df3.groupby("school size")
m_df_s= gdf_s.mean()
m_df_s= m_df_s.rename(columns={'avg_read':'Average Reading Score','avg_math':'Average Math Score'})
m_df_s.drop(['total students', 'total budget','budget per student'],axis=1,inplace=True)
m_df_s

,average math score,average reading score,% passing math,% passing reading,% Overall passing Rate
school size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


##  Scores by School Type

In [26]:
# calculate the average scores and the passing scores by the school type
gdf_t=df3.groupby("school type")
m_df_t= gdf_t.mean()
m_df_t= m_df_t.rename(columns={'avg_read':'Average Reading Score','avg_math':'Average Math Score'})
m_df_t.drop(['total students', 'total budget','budget per student'], axis=1, inplace=True)
m_df_t

,average math score,average reading score,% passing math,% passing reading,% Overall passing Rate
school type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
